# FIT5196 Assessment 1
#### Student Name:
#### Student ID:

Date: 08/15/2019

version: 1

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)
    
Libraries used:
* re 2.2.1 (for regular expression, included in Anaconda python 3.6)
* pandas 0.25.0 (for data frame, included in Anaconda python 3.6)


## 1.Introduction
this assignment comprises the extraction of specific content from the text text and processing the proposed file. There are 151 patents in the file named 'Group146.txt'. the assignment requirements are the following:

1. extracting several patent grants. e.g., patent title, patent ID, citation network, abstract etc.
2. transforming this data into CSV and JSON format.
3. removing the tags in the data.

More details for each task will be given in the following sections

## 2.Import libraries


In [1]:
import re
import pandas as pd

## 3.Functions
This assignment use 11 functions to segementation the data and extract the special information from the data.

### 3.1 Segementation function

In [2]:
def segmentation(file):
    result = re.findall('<\?xml([\d\D]*?)</us-patent-grant>',file)
    return result

From the format of dataset, we can recognize that each description of patent is begin with '' and end with ''. So we split the dataset so that each part can involve all informations of one patent. We use the code`'[\d\D]'` to capture everything between `<?xml` and `</us-patent-grant>`. Furthermore the `*?` can prevent the regular expression capturing the whole file.

### 3.2 The function of extracting the grant id

In [3]:
def find_grant_id(inputstring):
	grant_id = re.search('file="(.*)-\d*.XML"', inputstring)
	return grant_id.group(1)

This function is to get the ID of grand. From the dataset, we can easily konw ids are involved in name of file. 
So this expression captures the number between `file=` and `-\d*.XML`.(`-d*` can represent more than one digits)

### 3.3 The function of finding the title

In [4]:
def find_patent_title(inputstring):
	patent_title = re.search('<invention-title id=".*">(.*)</invention-title>', inputstring)
	return patent_title.group(1)

In this dataset, patent title follows invention-title ID. For the purpose of getting patent title, we use `.*` to 
find all characters expect `\n` between two tags.

### 3.4 The function of extracting the kind

In [5]:
def find_kind(inputstring):
	kind_id = re.search('<publication-reference>[\d\D]*?<kind>(.*)</kind>', inputstring)
	if kind_id.group(1) == 'B2':
		kind_result = 'Utility Patent Grant (with a published application) issued on or after January 2, 2001.'
		return kind_result
	elif kind_id.group(1) == 'S1':
		kind_result = 'Design Patent'
		return kind_result
	elif kind_id.group(1) == 'E1':
		kind_result = 'Reissue Patent'
		return kind_result
	elif kind_id.group(1) == 'B1':
		kind_result = 'Utility Patent Grant (no published application) issued on or after January 2, 2001.'
		return kind_result
	elif kind_id.group(1) == 'P3':
		kind_result = 'Plant Patent Grant (with a published application) issued on or after January 2, 2001'
		return kind_result
	elif kind_id.group(1) == 'P2':
		kind_result = 'Plant Patent Grant (no published application) issued on or after January 2, 2001'
		return kind_result

`<kind>B2</kind>`
There only has code of kind in dataset. So we should find the code first, then replace code with the specific content.
Because there are many tags `<kind>` in dataset and most of them are not what we want to extract, we add the condition
`<publication-reference>[\d\D]*?<kind>` in expression. The format of kind in the data is a letter plus a number, the function transform these kinds to a special string.

### 3.5 Counting the  number of claims

In [6]:
def find_the_number_of_claims(inputstring):
	number_of_claims = re.search(r'<number-of-claims>(\d*)</number-of-claims>', inputstring)
	return number_of_claims.group(1)

The number of claims exists between the two tags `<number-of-claims>` and `</number-of-claims>`, which can be easily 
to find out.

### 3.6 The function of extracting the name

In [7]:
def find_the_name(inputstring):
    inventors_name = ''
    inventors = re.findall(r'<inventor[\d\D]*?</inventor>', inputstring)
    for inventor in inventors:
        inventor_last_name = re.search(r'<last-name>(.*?)</last-name>', inventor)
        inventor_first_name = re.search(r'<first-name>(.*?)</first-name>', inventor)
        if inventor_last_name is None and inventor_first_name is None:
            inventor_name = 'NA'
        elif inventor_last_name is None:
            inventor_name = inventor_first_name.group(1)
        elif inventor_first_name is None:
            inventor_name = inventor_last_name.group(1)
        else:
            inventor_name = inventor_first_name.group(1) + ' ' + inventor_last_name.group(1)
        inventors_name = inventors_name + inventor_name + ','
    result = '[' + inventors_name[:-1] + ']'
    return result

At first, this function create a empty string (`inventors_name`) to store the inventor name. Each patent may have many inventors, and each inventor's name begin with tag `<inventor...>` end with `</inventor>`, so the regular expression is `r'<inventor([\d\D]*?)</inventor>'`. `[/d/D]` (chose number or non-number) causes to capture everything including line breaks and `*?` is non-greedy pattern which prevents matching of all data. `findall` put the result of matching into a list. Each element in the list is all information of each person. 

Function use `search` to match first name and last name for each inventor. The regular expressions are `r'<last-name>(.*?)</last-name>'` and `r'<first-name>(.*?)</first-name>'`. `.*?` means matching everything expect line breaks and uesing no-greedy pattern. `()` makes the content between `<last_name>` and `</last-name>` to be chosen and makes them be one group. 

Considering some special circumstances, such as some patents without inventors or inventors without first-name or last-name. Function added judgment statement. if the patent does not have inventor, the inventor name should be NA. if the inventor does not have first-name, the inventor name should be last-name, if the inventor does not have last-name, the result is reversed. When the function gets the name, the function add a , behind the name and add the name string with the `inventors_name`. After extracting all the names, remove the last comma and add [] before and after the string.

### 3.7 The function of counting applicant

In [8]:
def applicant_count(inputstring):
    count = re.findall(r'<category>.*applicant</category>', inputstring)
    return len(count)

This function use regular expression to find the applicants and put them into a list. the regular expression is `r'<category>.*applicant</category>'` (`.*` means match everything expect line breaks 0 or more times), and then put them into the list. The return value is the length of the list which is the number of all applicants.

### 3.8 The function of counting examiner

In [9]:
def examiner_count(inputstring):
    count = re.findall(r'<category>.*examiner</category>', inputstring)
    return len(count)

This function is same with counting applicant. This function put the matching result into the list and return the length of the list. The length is the number of all examiner.

### 3.9 The function of matching abstract

In [10]:
def find_abstract(inputstring):
    abstract_tag = re.search('<abstract id="abstract">\n([\s\S]*?)\n</abstract>',inputstring)
    if abstract_tag is None:
        return 'NA'
    else:
        clean_abstract_2 = re.sub('\n', ' ', abstract_tag.group(1)) # replace the break line symbol to space
        return clean_abstract_2

This function use regular expression (`'<abstract id="abstract">\n([\s\S]*?)\n</abstract>'`) to find the abstracts which are begin with `<abstract id="abstract">\n` end with `\n</abstract>'`. `\n` is line break symbol. `[\s\S]*?` is matching space or non-space and use the non-greedy pattern. Function uses a conditional statement. When the regular expression can not find the abstract, the function will return a string 'NA'. `clean_abstract_2 = re.sub('\n', ' ', abstract_tag.group(1))` this common can replace the break line symbol to space.

### 3.10 The function of matching claim

In [11]:
def find_claim_text(inputstring):
    claim_text = re.findall('<claims id="claims">\n([\d\D]*?)\n</claims>', inputstring)
    if claim_text is None:
        return 'NA'
    else:
        clean_claim_text = ''
        for i in claim_text:
            clean_claim_text = clean_claim_text + i + ','
        clean_claim_text_3 = re.sub('\n', ' ', clean_claim_text)
        clean_claim_text_4 = '[' + clean_claim_text_3[:-1] + ']'
        return clean_claim_text_4

The regular expression is same as the function of matching abstract. `<claims id="claims">\n([\d\D]*?)\n</claims>` to find the claims. `[\d\D]*?` means that chosing number or non-number and with non-greedy pattern. When the regular expression can not find the abstract, the finction will return a string 'NA'. if the function match the claim, `findall` will return a list to store the claims. This function make this list become string that meets the requirements, and then return the string.

### 3.11 Transfer the data into JSON format

In [12]:
def writting_json (string_1, string_2,string_3, string_4, string_5, string_6, string_7, string_8, string_9):
	result = '"'+string_1+'":{"'+\
		'patent_title'+'":"'+string_2+'","'+\
		'kind'+'":"'+string_3+'","'+\
		'number_of_claims'+'":'+str(string_4)+',"'+\
		'inventors'+'":"'+string_5+'","'+\
		'citations_applicant_count'+'":'+str(string_6)+',"'+\
		'citations_examiner_count'+'":'+str(string_7)+',"'+\
		'claims_text'+'":"'+string_8+'","'+\
		'abstract'+'":"'+string_9+'"},'
	return result

This function transfer data into JSON format. string_1 is grant_id, string_2 is patent_title, string_3 is kind, string_4 is the number of claims, string_5 is inventors' name, string_6 is the number of applicant, string_7 is the number of examiner, string_8 is claims text, string_9 is abstract. Function adds these variables to a specific string, making them a special format.

### 3.12 The function of cleanning the tag

In [13]:
def clean_tag (inputstring):
	tags = re.findall('</(.*?)>', inputstring)
	for tag in set(tags):
		pattern = '<' + tag + '.*?>'
		reg = re.compile(pattern)
		inputstring = reg.sub('', inputstring)
	inputstring = re.sub(r'</.*?>', '', inputstring)
	inputstring = re.sub(r'<\?.*?\?>','', inputstring)
	inputstring = re.sub(r'<.*?/>', '', inputstring)
	return inputstring

This function can clean the tag inside the data, such as, `<p>`, `</p>`, `<?...?>`, `<.../>`. At first we find the content between `<` and `>`, and according to these tag's contents, to find the tag like `<..>`. because we need to avoid to delete biger than and less than symbol 

## 4.Main function

In [23]:
file = open('Group146.txt').read()
json_file = open('Group146.json', 'w')
json = ''
df = pd.DataFrame(columns = ['grant_id', 'patent_title','kind', 'number_of_claims', 'inventors', 'citations_applicant_count', 'citations_examiner_count', 'claims_text', 'abstract'])
content = segmentation(file)
print(content)
for part in content:
    grant_id = find_grant_id(part)
    print(grant_id)
    patent_title = find_patent_title(part)
    print(patent_title)
    kind = find_kind(part)
    print(kind)
    number_of_claims = find_the_number_of_claims(part)
    print(number_of_claims)
    inventors = find_the_name(part)
    print(inventors)
    citation_applicant_count = applicant_count(part)
    print(citation_applicant_count)
    citations_examiner_count = examiner_count(part)
    print(citations_examiner_count)
    claims_text = clean_tag(find_claim_text(part))
    print(claims_text)
    abstract = clean_tag(find_abstract(part))
    print(abstract)
    json_row = writting_json(grant_id, patent_title, kind, number_of_claims, inventors, citation_applicant_count, citations_examiner_count, claims_text, abstract)
    json += json_row
    df = df.append(pd.DataFrame({'grant_id': [grant_id], 'patent_title':[patent_title],'kind':[kind], 'number_of_claims':[number_of_claims], 'inventors':[inventors], 'citations_applicant_count':[citation_applicant_count], 'citations_examiner_count':[citations_examiner_count], 'claims_text':[claims_text], 'abstract':[abstract]}), ignore_index=True)
df.to_csv('Group146.csv', index=False)
json_content = '{'+json[:-1]+'}'
json_file.write(json_content)
json_file.close()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




US10358615
Method for improving low temperature stability of a friction modifier additive
Utility Patent Grant (with a published application) issued on or after January 2, 2001.
12
[Theresa Liang Gunawan]
3
5
[ 1. A method comprising the steps of (a) receiving a fuel additive solution consisting of (i) from about 5 to about 50 wt. %, based on the total weight of the fuel additive solution, of one or more fully saturated fatty acid sorbitan esters selected from the group consisting of sorbitan monolaurate, sorbitan monopalmitate, sorbitan monostearate, and sorbitan monoisostearate, (ii) about 10 to about 95 wt. %, based on the total weight of the fuel additive solution, of a hydrocarbon fluid, and (iii) optionally, one or more fuel additives selected from the group consisting of a detergent, cetane improver, antioxidant, metal deactivator, dye, marker, corrosion inhibitor, antistatic additive, drag reducing agent, demulsifier, dehazer, anti-icing additive, lubricity additives, combusti

[ 1. A method of scheduling a semiconductor substrate processing sequence in an integrated substrate processing system, comprising: assigning a processing sequence to each semiconductor substrate in a batch of semiconductor substrates to be processed; for each processing chamber in the integrated substrate processing system, assigning a processing chamber to each process in the processing sequence; generating a processing model for the batch of semiconductor substrates, wherein the processing model defines a start time for each substrate in each processing chamber; generating a timetable for the batch of semiconductor substrates based off the processing model; and processing the batch of semiconductor substrates in accordance with the timetable.    2. The method of claim 1, wherein the processing sequence is the same for each semiconductor substrate in the batch.   3. The method of claim 1, wherein the processing sequence is not the same for each semiconductor substrate in the batch.  

US10361250
Active optical device enabled by dielectric metamaterials
Utility Patent Grant (with a published application) issued on or after January 2, 2001.
12
[Sheng Liu,Igal Brener,Michael B. Sinclair]
28
4
[ 1. An active optical apparatus comprising a substrate and an optical metamaterial, wherein: the optical metamaterial comprises a regular two-dimensional array of vertical columns overlying the substrate; each of the vertical columns comprises a layered structure of semiconductor material; each said layered structure includes a quantum-well multilayer and has an effective refractive index; each said layered structure is underlain by an optical confinement layer; each said optical confinement layer comprises a confinement material that has a refractive index lower than the effective refractive index of the layered semiconductor structure that overlies it; each of the vertical columns is conformed to provide at least one dielectric resonator having an optical resonance at a frequen

USD0854761
Animal waste removal apparatus
Design Patent
1
[Mike Filmer]
8
150
[ The ornamental design for an animal waste removal apparatus, as shown and described. ]
NA
US10359222
Controller for a vapour compression system and a method for controlling a vapour compression system
Utility Patent Grant (with a published application) issued on or after January 2, 2001.
10
[Roozbeh Izadi-Zamanabad,Harald Alnor]
13
10
[ 1. A controller for a vapour compression system for cooling a refrigerated space, the system comprising a circuit for circulation of a refrigerant between a compressor, a condenser, and an evaporator, and an expansion valve controlling a flow of the refrigerant into the evaporator and thereby cooling of the refrigerated space, wherein the controller is adapted to control the expansion valve to cool the refrigerated space based on a first temperature in the circuit after the evaporator and before the compressor and a second air temperature determined in the refrigerated space

This is the main function, at first, the function use `open` command to opean the data file named 'Group146.txt' and create a JSON file named 'Group146.json', and then use `df = pd.DataFrame` to create a empty data frame. `content = segmentation(file)` use the function 
* `segmentation()` to segment the file.

* `grant_id = find_grant_id(part)` use function `find_grant_id()` to get the grant id from the data.

* `patent_title = find_patent_title(part)` use function `fing_patent_title()` to get the title.

* `kind = find_kind(part)` use function `find_kind()` to get the patent's kind.

* `number_of_claims = find_the_number_of_claims(part)` use function `find_the_number_of_claims()` to get the number of claims.

* `inventors = find_the_name(part)` use function `find_the_name()` to get the inventors' name.

* `citation_applicant_count = applicant_count(part)` use function `applicant_count()` to count the number of applicant.

* `citations_examiner_count = examiner_count(part)` use function `examiner_count()` to count the number of examiner.

* `claims_text = clean_tag(find_claim_text(part))` use function `find_claim_text()` to find the claim text and then use function 

* `clean_tag()` to clean tag inside the claim text.

* `abstract = clean_tag(find_abstract(part))` use function `find_abstract()` to find the abstract and then use function

* `clean_tag()` to clean tag inside the abstract.

The function use `writting_json` to add these information together and transform them into JSON format, and store them into the variable `json`. function use `df.append` to append this information into the data frame. `json_content = '{'+json[:-1]+'}'` is to add { and } at the beginning and end, and remove the last ','. 

